# Generate a Crosswalk with Risk Atlas Nexus

## Dependencies
Tip: Ensure you have followed installation instructions for the risk_atlas_nexus library

```
git clone git@github.com:IBM/risk-atlas-nexus.git
cd risk-atlas-nexus
python -m venv vrisk-atlas-nexus
source vrisk-atlas-nexus/bin/activate
pip install -e .
```

In [1]:
# imports
from rich import print
from numpy import nan
from collections import defaultdict
import pandas as pd
from pydantic import BaseModel

from risk_atlas_nexus.ai_risk_ontology.datamodel.ai_risk_ontology import Risk
from risk_atlas_nexus import RiskAtlasNexus

# Set the option to display full column width
pd.set_option('display.max_colwidth', None)


/Users/ingevejs/Documents/workspace/ingelise/risk-atlas-nexus/src/risk_atlas_nexus/toolkit/job_utils.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Crosswalk documents are intended to provide a mapping of concepts and terms between the different taxonomies, frameworks, standards and regulation documents. 

We will use the Risk Atlas Nexus library to generate a crosswalk document between two of the available taxonomies.


In [2]:
# Understand what taxonomies are available to us
ran = RiskAtlasNexus() # no args, so default configuration 
print(f"\n# Total risks available : {len(ran.get_all_risks())}") # 560
print(f"\n# Available taxonomies : {len(ran.get_all_taxonomies())}") # 10
print(f"\n# Available taxonomy IDs : {[x.id for x in ran.get_all_taxonomies()]}")

[2025-07-03 09:57:16:457] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None


# Total risks available : 560

# Available taxonomies : 10

# Available taxonomy IDs : ['ibm-risk-atlas', 'nist-ai-rmf', 'ailuminate-v1.0', 'mit-ai-risk-repository', 
'mit-ai-risk-repository-causal', 'ai-risk-taxonomy', 'cset', 'ibm-granite-guardian', 'owasp-llm-2.0', 'credo-ucf']

For this example, we want to produce content for a crosswalk between the risks in the NIST AI RMF and the IBM AI risk Atlas.  We will specify these as the target taxonomies, and then find the related risks and display a subset of the risk content in a pandas dataframe.

In [3]:
# declare target taxonomies
taxonomy_1 = 'nist-ai-rmf'
taxonomy_2 = 'ibm-risk-atlas'

# Helper functions

A few utility functions to format the crosswalk dataframe output are provided below.

In [ ]:

class RiskSimplified(BaseModel):
    id: str
    name: str
    isPartOf: str | None
    isDefinedByTaxonomy: str | None


def expand_risks(list_of_risk_ids):
    """Retrieve content for the related risk IDs and return a string

    Args:
        list_of_risk_ids: List[str]
    
    Return:
        str 
            Result containing Formatted string for the dataframe        
    """
    risks = [item for item in (ran.get_risk(id=risk_id) for risk_id in list_of_risk_ids if risk_id is not None) if item is not None]
    grouped_risks = defaultdict(list)
    
    for risk in risks:
        selected_fields_risk = RiskSimplified(id=risk.id, name=risk.name, isPartOf=risk.isPartOf, isDefinedByTaxonomy=risk.isDefinedByTaxonomy)
        grouped_risks[risk.isDefinedByTaxonomy].append(selected_fields_risk)
        
    return str(dict(grouped_risks))

  
def construct_crosswalk_df(taxonomy_1, taxonomy_2):
    """Construct a crosswalk dataframe

    Args:
        taxonomy_1: str
            taxonomy identifier
        taxonomy_2: str
            taxonomy identifier

    Return:
        pd.DataFrame 
            Results in form of a pd.Dataframe      
    
    """
    # get all risks from taxonomy 1 into a dataframe
    df = pd.DataFrame([res.model_dump() for res in ran.get_all_risks(taxonomy=taxonomy_1)])

    # we only need to show a subset of columns
    df = df[['id', 'name', 'description', 'hasRelatedAction', 'isPartOf',
       'closeMatch', 'exactMatch', 'broadMatch', 'narrowMatch',
       'relatedMatch']]
    
    # show only risk content relating to the second taxonomy
    def filter_by_taxonomy(list_risks):
        if list_risks is not None:
            return [id for id in list_risks if ran.get_risk(id=id) is not None and ran.get_risk(id=id).isDefinedByTaxonomy == taxonomy_2]
        else:
            return None

    df['closeMatch'] =  df['closeMatch'].apply(filter_by_taxonomy)
    df['exactMatch'] = df['exactMatch'].apply(filter_by_taxonomy)
    df['broadMatch'] = df['broadMatch'].apply(filter_by_taxonomy)
    df['narrowMatch'] = df['narrowMatch'].apply(filter_by_taxonomy)
    df['relatedMatch'] = df['relatedMatch'].apply(filter_by_taxonomy)
    df.rename(columns={"id": "RAN ID"})
    

    df['expanded_risks'] = df.apply(
        lambda row: expand_risks((row['closeMatch'] or []) + (row['exactMatch'] or []) + (row['broadMatch'] or []) + (row['narrowMatch'] or []) + (row['relatedMatch'] or [])), axis=1
    )

    return df

df = construct_crosswalk_df(taxonomy_1, taxonomy_2)
df

,id,name,description,hasRelatedAction,isPartOf,closeMatch,exactMatch,broadMatch,narrowMatch,relatedMatch,expanded_risks
0,nist-cbrn-information-or-capabilities,CBRN Information or Capabilities,"Eased access to or synthesis of materially nefarious information or design capabilities related to chemical, biological, radiological, or nuclear (CBRN) weapons or other dangerous materials or agents.","[GV-1.2-002, GV-1.3-001, GV-1.3-002, GV-1.3-003, GV-1.3-004, GV-1.4-002, GV-2.1-004, GV-2.1-005, GV-3.2-001, GV-3.2-005, MP-1.1-004, MP-4.1-005, MP-4.1-008, MP-5.1-004, MS-1.1-004, MS-1.1-005, MS-1.1-008, MS-1.3-001, MS-1.3-002, MS-2.3-004, MS-2.6-002, MS-2.6-006, MS-2.6-007, MG-2.2-001, MG-2.2-005, MG-3.1-004, MG-3.2-009, MG-4.1-002]",None,[],None,None,[atlas-dangerous-use],[atlas-harmful-output],"{'ibm-risk-atlas': [RiskSimplified(id='atlas-dangerous-use', name='Dangerous use', isPartOf='ibm-risk-atlas-misuse', isDefinedByTaxonomy='ibm-risk-atlas'), RiskSimplified(id='atlas-harmful-output', name='Harmful output', isPartOf='ibm-risk-atlas-value-alignment', isDefinedByTaxonomy='ibm-risk-atlas')]}"
1,nist-confabulation,Confabulation,The production of confidently stated but erroneous or false content (known colloquially as “hallucinations” or “fabrications”) by which users may be misled or deceived.,"[GV-1.3-002, GV-4.1-001, GV-5.1-002, MS-2.3-001, MS-2.3-002, MS-2.3-004, MS-2.5-001, MS-2.5-003, MS-2.6-005, MS-2.9-001, MS-2.13-001, MS-3.2-001, MS-4.2-002, MG-2.2-009, MG-3.2-009, MG-4.1-002, MG-4.1-004, MG-4.3-002]",None,None,[atlas-hallucination],None,None,None,"{'ibm-risk-atlas': [RiskSimplified(id='atlas-hallucination', name='Hallucination', isPartOf='ibm-risk-atlas-robustness', isDefinedByTaxonomy='ibm-risk-atlas')]}"
2,nist-dangerous-violent-or-hateful-content,"Dangerous, Violent, or Hateful Content","Eased production of and access to violent, inciting, radicalizing, or threatening content as well as recommendations to carry out self-harm or conduct illegal activities. Includes difficulty controlling public exposure to hateful and disparaging or stereotyping content.","[GV-1.3-001, GV-1.3-002, GV-1.3-004, GV-1.3-006, GV-1.4-001, GV-2.1-004, GV-2.1-005, GV-4.2-001, MP-1.1-003, MP-1.1-004, MP-3.4-006, MP-4.1-005, MP-4.1-008, MP-5.1-002, MP-5.1-004, MS-2.2-002, MS-2.3-004, MS-2.5-006, MS-2.6-001, MS-2.6-002, MS-2.6-003, MS-2.6-004, MS-2.7-007, MS-2.7-008, MS-2.11-002, MS-2.12-001, MG-2.2-001, MG-2.2-005, MG-3.2-005, MG-4.2-002]",None,"[atlas-harmful-output, atlas-toxic-output]",None,None,[atlas-harmful-code-generation],[],"{'ibm-risk-atlas': [RiskSimplified(id='atlas-harmful-output', name='Harmful output', isPartOf='ibm-risk-atlas-value-alignment', isDefinedByTaxonomy='ibm-risk-atlas'), RiskSimplified(id='atlas-toxic-output', name='Toxic output', isPartOf='ibm-risk-atlas-value-alignment', isDefinedByTaxonomy='ibm-risk-atlas'), RiskSimplified(id='atlas-harmful-code-generation', name='Harmful code generation', isPartOf='ibm-risk-atlas-value-alignment', isDefinedByTaxonomy='ibm-risk-atlas')]}"
3,nist-data-privacy,Data Privacy,"Impacts due to leakage and unauthorized use, disclosure, or de-anonymization of biometric, health, location, or other personally identifiable information or sensitive data.","[GV-1.1-001, GV-1.2-001, GV-1.4-002, GV-1.6-003, GV-4.3-003, GV-6.1-001, GV-6.1-005, GV-6.1-009, GV-6.2-003, MP-1.1-001, MP-2.1-002, MP-4.1-001, MP-4.1-003, MP-4.1-004, MP-4.1-005, MP-4.1-008, MP-4.1-009, MP-4.1-010, MS-1.3-003, MS-2.2-002, MS-2.2-003, MS-2.2-004, MS-2.3-004, MS-2.6-002, MS-2.7-001, MG-2.2-009, MG-3.1-002, MG-3.2-002, MG-4.3-003]",None,None,None,None,"[atlas-attribute-inference-attack, atlas-data-privacy-rights, atlas-data-usage-rights, atlas-exposing-personal-information, atlas-ip-information-in-prompt, atlas-legal-accountability, atlas-membership-inference-attack, atlas-model-usage-rights, atlas-nonconsensual-use, atlas-personal-information-in-data, atlas-personal-information-in-prompt, atlas-reidentification]",[atlas-harmful-ou